Gianmarco Alessio - 2024/02/22

# AI Developer Junior – Test Medium Level [2] 

Progetto  di  Elaborazione  del  Linguaggio  Naturale  (NLP)  con  Python  e  Scikit-
Learn 

**Descrizione  del  progetto**:  Hai  a  disposizione  un  dataset  contenente  recensioni  di 
film e le relative etichette di sentiment, ovvero se una recensione è positiva o negativa. 
Il tuo obiettivo è sviluppare un modello di machine learning per la classificazione dei 
sentiment  delle  recensioni  utilizzando  tecniche  di  NLP.  Deve  essere  in  grado  di 
raggiungere un'accuratezza di almeno l’85% nella classificazione dei sentiment. 
Compiti per il candidato: 

- Esplorazione dei dati: iniziare con un'analisi esplorativa dei dati per 
comprendere  la  distribuzione  delle  etichette  di  sentiment,  la  lunghezza  delle 
recensioni, ecc. 
- Preelaborazione  dei  dati:  eseguire  la  preelaborazione  dei  dati,  inclusa  la 
rimozione di stopwords, la tokenizzazione delle frasi, la creazione di vettori di 
parole (word embeddings) e la suddivisione del dataset in set di addestramento 
e di test. 
- Sviluppo del modello: progettare e allenare un modello di machine learning 
per la classificazione dei sentiment utilizzando Scikit-Learn. Puoi sperimentare 
con algoritmi come Support Vector Machines (SVM), Naive Bayes, o modelli di 
deep  learning  come  reti  neurali  ricorrenti  (RNN)  o  Long  Short-Term  Memory 
(LSTM) se si sentono confortevoli. 
- Valutazione del modello: valutare le prestazioni del modello utilizzando il set 
di test e misurare l'accuratezza, la precisione, il richiamo e l'F1-score. 
- Ottimizzazione  del  modello:  Se  il  modello  iniziale  non  raggiunge  l'85%  di 
accuratezza, cercare di ottimizzarlo. Puoi eseguire l'ottimizzazione dei 
parametri, provare diverse configurazioni di algoritmi o esplorare l'uso di word 
embeddings pre-addestrati. 
- Presentazione  dei  risultati:  presentare  i  risultati  del  progetto  e  spiegare  il 
processo seguito per sviluppare il modello. 
 


Importazione librerie utilizzate:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from scipy.sparse import csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn


/var/folders/wq/6qt2cn694p9fgrldr8r274tm0000gn/T/ipykernel_18264/4187975235.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Caricamento del dataset chiamato `aclImdb` contenente recensioni di film e le relative etichette di sentiment, ovvero se una recensione è positiva o negativa:

In [2]:
def load_reviews(directory):
    reviews = []
    labels = []
    for label_type in ['pos', 'neg']:
        dir_name = os.path.join(directory, label_type)
        for file in os.listdir(dir_name):
            if file.endswith('.txt'):
                file_path = os.path.join(dir_name, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    reviews.append(f.read())
                    # 'pos' corrisponda a recensioni positive (etichetta = 1)
                    # e 'neg' a recensioni negative (etichetta = 0)
                    labels.append(1 if label_type == 'pos' else 0)
    return reviews, labels


train_reviews, train_labels = load_reviews('aclImdb/train/')
test_reviews, test_labels = load_reviews('aclImdb/test/')


# 1) Esplorazione dei dati

- Distribuzione delle etichette di sentiment (50% positiva e 50% negativa)
- Frequenza delle lunghezze delle recensioni

In [ ]:
# Converti in DataFrame
train_data = pd.DataFrame({
    'review': train_reviews,
    'label': train_labels
})

test_data = pd.DataFrame({
    'review': test_reviews,
    'label': test_labels
})

# Funzioni per analizzare i dati
def analyze_data(data, set_name="Train"):
    print(f"Analisi del set {set_name}")
    print(data['label'].value_counts(normalize=True))
    data['length'] = data['review'].apply(len)
    plt.figure(figsize=(10, 6))
    plt.hist(data['length'], bins=50, alpha=0.7)
    plt.title(f'Distribuzione della Lunghezza delle Recensioni - {set_name}')
    plt.xlabel('Lunghezza della Recensione')
    plt.ylabel('Frequenza')
    plt.show()

# Since we already have the word frequencies, we can directly generate the word clouds

def create_word_cloud(frequencies, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(frequencies)
    
    # Plot
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Funzione per plottare l'istogramma delle parole più frequenti
def plot_word_frequencies(word_freq, title,top_n=100):
    words = list(word_freq.keys())
    frequencies = list(word_freq.values())
    
    # Ordinamento basato sulle frequenze
    indices_sorted = np.argsort(frequencies)[::-1]
    words_sorted = np.array(words)[indices_sorted][:top_n]
    frequencies_sorted = np.array(frequencies)[indices_sorted][:top_n]
    
    plt.figure(figsize=(10, 8))
    plt.bar(words_sorted, frequencies_sorted)
    plt.title(title)
    plt.xticks(rotation=90)
    plt.xlabel('Words')
    plt.ylabel('Frequencies')
    plt.show()

# Function to load the labeled Bow features from the .feat file
def load_labeled_bow(file_path):
    data = []
    indices = []
    indptr = [0]
    labels = []

    # Read the file
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            elements = line.split()
            labels.append(int(elements[0]))
            for item in elements[1:]:
                index, value = item.split(':')
                indices.append(int(index))
                data.append(int(value))
            indptr.append(len(indices))

    # Create the CSR matrix
    labeled_bow_matrix = csr_matrix((data, indices, indptr), dtype=int)
    return labels, labeled_bow_matrix

# Function to calculate word frequencies based on the BoW matrix and labels
def calculate_word_frequencies(bow_matrix, labels, vocab):
    # Split the matrix into positive and negative reviews based on labels
    positive_matrix = bow_matrix[labels == 1]
    negative_matrix = bow_matrix[labels == -1]
    
    # Sum up the word counts. csr_matrix automatically sums along axis 0 when using np.sum
    positive_word_counts = np.sum(positive_matrix, axis=0)
    negative_word_counts = np.sum(negative_matrix, axis=0)
    
    # Convert the matrix row to a flat array
    positive_word_counts = np.array(positive_word_counts).flatten()
    negative_word_counts = np.array(negative_word_counts).flatten()
    
    # Map indices to words using the provided vocabulary
    positive_word_freq = {vocab[i]: count for i, count in enumerate(positive_word_counts) if count > 0}
    negative_word_freq = {vocab[i]: count for i, count in enumerate(negative_word_counts) if count > 0}
    
    # Sort the words based on frequency
    positive_word_freq = dict(sorted(positive_word_freq.items(), key=lambda item: item[1], reverse=True))
    negative_word_freq = dict(sorted(negative_word_freq.items(), key=lambda item: item[1], reverse=True))
    
    return positive_word_freq, negative_word_freq


In [ ]:
# Esegui l'analisi per i dati di addestramento e di test
analyze_data(train_data, "Addestramento")
analyze_data(test_data, "Test")

Rimozione delle stopwords e tokenizzazione delle frasi per la rappresentazione della cloud word. Facendo uso delle librerie `nltk` e `wordcloud`, e `imdb.vocab` e `labeledBow.feat` per la rimozione delle stopwords.

In [ ]:
vocab_path = 'aclImdb/imdb.vocab'
labeled_bow_path = 'aclImdb/train/labeledBow.feat'

with open(vocab_path, 'r', encoding='utf-8') as file:
    vocab = [line.strip() for line in file.readlines()]

# Load the data
labels, bow_matrix = load_labeled_bow(labeled_bow_path)

# Check the shape of the matrix and the distribution of labels
bow_matrix_shape = bow_matrix.shape
labels_array = np.array(labels)
adjusted_labels = np.where(labels_array < 5, -1, 1)

# Count the occurrences of each sentiment
positive_reviews_count = (adjusted_labels == 1).sum()
negative_reviews_count = (adjusted_labels == -1).sum()

(bow_matrix_shape, positive_reviews_count, negative_reviews_count)


In [ ]:
# Define the number of top words to consider
top_n = 10 # create a plot for the different accuracies values in relationships of the words we consider 
positive_word_freq, negative_word_freq = calculate_word_frequencies(bow_matrix, adjusted_labels, vocab)

# Download stopwords from NLTK
nltk.download('stopwords')

# Get English stopwords
stop_words = set(stopwords.words('english'))

# Filter out stopwords from the frequencies for more meaningful word clouds
positive_word_freq_filtered = {word: freq for word, freq in positive_word_freq.items() if word not in stop_words and freq > 10}
negative_word_freq_filtered = {word: freq for word, freq in negative_word_freq.items() if word not in stop_words and freq > 10}

# Find common words between the top N words of both positive and negative reviews
common_words = set(list(positive_word_freq_filtered.keys())[:top_n]).intersection(set(list(negative_word_freq_filtered.keys())[:top_n]))

# Remove common words from the filtered frequencies
positive_word_freq_filtered = {word: freq for word, freq in positive_word_freq_filtered.items() if word not in common_words}
negative_word_freq_filtered = {word: freq for word, freq in negative_word_freq_filtered.items() if word not in common_words}

# Generate and display the word clouds for the top words in positive and negative reviews
create_word_cloud(positive_word_freq_filtered, 'Word Cloud for Positive Reviews')
create_word_cloud(negative_word_freq_filtered, 'Word Cloud for Negative Reviews')

# 2) Preelaborazione dei dati

- Rimozione delle stopwords
- Tokenizzazione delle frasi
- Creazione di vettori di parole (word embeddings)
- Suddivisione del dataset in set di addestramento e di test (già effettuata)

In [3]:
# Assumendo che `reviews` sia la tua lista di recensioni e `sentiments` le etichette di sentiment corrispondenti
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')

# 3) Sviluppo del modello

- Progettazione e allenamento di un modello di machine learning per la classificazione dei sentiment utilizzando Scikit-Learn.
- Sperimentazione con algoritmi come Support Vector Machines (SVM), Naive Bayes, o modelli di deep learning come reti neurali ricorrenti (RNN) o Long Short-Term Memory (LSTM).

In [ ]:
X = tfidf_vectorizer.fit_transform(train_reviews)
X_test = tfidf_vectorizer.transform(test_reviews)

y = train_labels
y_test = test_labels

## Allenamento con Naive Bayes

In [4]:
model = MultinomialNB()
model.fit(X, y)

Accuracy: 0.83444
              precision    recall  f1-score   support

           0       0.81      0.87      0.84     12500
           1       0.86      0.80      0.83     12500

    accuracy                           0.83     25000
   macro avg       0.84      0.83      0.83     25000
weighted avg       0.84      0.83      0.83     25000



## Valutazione di Naive Bayes

In [ ]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Alleanmento con Support Vector Machines (SVM)

In [ ]:
model_svm = SVC(kernel='linear')
model_svm.fit(X, y)

## Valutazione di Support Vector Machines (SVM)

In [ ]:
y_pred_svm = model_svm.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

## Conversione dei dati in tensori PyTorch per la rete neurale riccorente RNN

In [5]:
# Conversione dei dati in tensori PyTorch

X_train_tensor = torch.tensor(X.toarray(), dtype=torch.long)
y_train_tensor = torch.tensor(y, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Creazione di un DataLoader per l'addestramento
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

Definizione del modello RNN con 2 layer LSTM e 1 layer fully connected.

In [6]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        super(SentimentRNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        
        # Prendiamo l'output dell'ultimo time step
        output = self.fc(output[:, -1, :])
        output = self.sigmoid(output)
        
        return output

Definizione dei parametri di addestramento.

In [7]:
# Definizione dei parametri
vocab_size = 10000  # Dimensione del vocabolario
output_size = 1  # Output binario (positivo/negativo)
embedding_dim = 400  # Dimensione degli embeddings
hidden_dim = 256  # Dimensione dello stato nascosto
n_layers = 2  # Numero di layers RNN

model = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

# Funzione di perdita e ottimizzatore
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Ciclo di addestramento
epochs = 5


## Allenamento del modello RNN

In [8]:
for epoch in range(epochs):
    for inputs, labels in train_loader:
        model.zero_grad()
        
        output = model(inputs)
        loss = loss_fn(output.squeeze(), labels)
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

KeyboardInterrupt: 

## Valutazione di RNN

In [ ]:
# Valutazione
model.eval()  # Imposta il modello in modalità valutazione
accuracy = []

with torch.no_grad():
    for inputs, labels in test_loader:
        output = model(inputs)
        predictions = torch.round(output.squeeze())  # Arrotonda l'output a 0 o 1
        correct = (predictions == labels).float()  # Confronta con le etichette vere
        accuracy.append(correct.mean().item())

print(f"Accuracy: {np.mean(accuracy)}")